In [1]:
import os
import pandas as pd
import re
from scipy.stats.mstats import gmean

## Read in data

In [2]:
# get the path to the folder with the results
instance_fldr = os.path.join("results/bm23")

# map generator names to the corresponding data frames
df_map = {
    "None": pd.DataFrame(),
    "Farkas": pd.DataFrame(),
    "New": pd.DataFrame(),
    "Old": pd.DataFrame()
}

In [3]:
regex = re.compile(r'([a-zA-Z0-9-]+)_([a-z]+)_([0-9-]+)_([0-9]+)_([a-zA-Z ]+)')

# iterate over all files in the folder
for file_name in os.listdir(instance_fldr):
    
    # get the experimental set up
    match = regex.search(file_name)
    instance_name = match.group(1)
    perturbation = match.group(2)
    degree = 2**int(match.group(3))
    terms = int(match.group(4))
    generator = match.group(5)
    
    # read the file
    file_pth = os.path.join(instance_fldr, file_name)
    df = pd.read_csv(file_pth, keep_default_na=False)
    
    # add some identifying columns
    df["problem number"] = df.index
    df["instance"] = instance_name
    df["perturbation"] = perturbation
    df["degree"] = degree
    df["terms"] = terms
    
    # append to the appropriate data frame
    df_map[generator] = pd.concat([df_map[generator], df])

In [4]:
# merge the 4 different data frames into one
join_cols = ["instance", "perturbation", "degree", "terms", "problem number"]
df = df_map["None"].merge(df_map["New"], on=join_cols,
                                suffixes=(" None", None))
df = df.merge(df_map["Old"], on=join_cols,
                          suffixes=(" New", None))
df = df.merge(df_map["Farkas"], on=join_cols,
                          suffixes=(" Old", " Farkas"))
df

KeyError: 'instance'

In [21]:
def gap_closed(df, col):
    return (df[col] - df["lpBound None"]) / \
        (df['primalBound None'] - df["lpBound None"]) 

In [22]:
# find the optimality gap closed by each generator
df["Disjunction (New)"] = gap_closed(df, "disjunctiveDualBound New")
df["Disjunction (Old)"] = gap_closed(df, "disjunctiveDualBound Old")
df["VPCs (New)"] = gap_closed(df, "lpBoundPostVpc New")
df["VPCs (Old)"] = gap_closed(df, "lpBoundPostVpc Old")
df["VPCs (Farkas)"] = gap_closed(df, "lpBoundPostVpc Farkas")
df["Root Cuts (None)"] = gap_closed(df, "rootDualBound None")
df["Root Cuts (New)"] = gap_closed(df, "rootDualBound New")
df["Root Cuts (Old)"] = gap_closed(df, "rootDualBound Old")
df["Root Cuts (Farkas)"] = gap_closed(df, "rootDualBound Farkas")

In [23]:
# set aside core columns and filter for all subsequent dataframes
group_cols = ["instance", "perturbation", "degree", "terms"]
id_cols = ["problem number"]
mask = (df["problem number"] > 0) & (df["Disjunction (New)"] < .9999)

## Make bound table

In [24]:
# additional filtering for dataframe on bounds
fields = ["Disjunction (New)", "Disjunction (Old)", "VPCs (New)", "VPCs (Old)",
          "VPCs (Farkas)", "Root Cuts (None)", "Root Cuts (New)", "Root Cuts (Old)",
          "Root Cuts (Farkas)"]
bound_df = df.loc[mask, group_cols + id_cols + fields]
bound_df

,instance,perturbation,degree,terms,problem number,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Old),VPCs (Farkas),Root Cuts (None),Root Cuts (New),Root Cuts (Old),Root Cuts (Farkas)
5,bm23,rhs,2,64,5,0.784825,0.732415,0.757973,0.726031,0.000000,0.413128,0.758680,0.728494,0.413128
12,bm23,rhs,2,8,1,0.136666,0.076904,0.130857,0.076904,0.076904,0.364447,0.364447,0.364447,0.364447
13,bm23,rhs,2,8,2,0.233139,0.233139,0.231129,0.230013,0.182538,0.451858,0.451858,0.451858,0.451858
14,bm23,rhs,2,8,3,0.163926,0.163926,0.163926,0.163926,0.163926,0.376273,0.376273,0.376273,0.376273
15,bm23,rhs,2,8,4,0.283649,0.283649,0.282967,0.283649,0.192381,0.436065,0.436082,0.436078,0.436073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,bm23,objective,4,16,6,0.437880,0.354568,0.374561,0.299250,0.342344,0.312068,0.379756,0.313838,0.369199
378,bm23,objective,4,16,7,0.395430,0.395430,0.372075,0.380370,0.382570,0.348786,0.390846,0.396553,0.389163
379,bm23,objective,4,16,8,0.395430,0.395430,0.380581,0.395430,0.384407,0.356923,0.400758,0.405044,0.401618
380,bm23,objective,4,16,9,0.476621,0.420377,0.462353,0.417436,0.397631,0.379600,0.465689,0.439910,0.409087


In [25]:
aggregations = {f: "mean" for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

In [26]:
# get gap closed by degree and term
bound_df.groupby(["degree", "terms"]).agg(aggregations)

Disjunction (New)  Disjunction (Old)  VPCs (New)  VPCs (Old)  \
degree terms                                                                 
1      4               0.147852           0.156649    0.147852    0.156649   
       8               0.194161           0.190213    0.193023    0.189511   
       16              0.412469           0.418550    0.395202    0.396094   
       32              0.593807           0.601539    0.581627    0.595258   
       64              0.705942           0.716947    0.676154    0.699999   
2      4               0.132532           0.148142    0.132532    0.148142   
       8               0.189254           0.184161    0.187232    0.183085   
       16              0.425689           0.405262    0.399896    0.389040   
       32              0.576933           0.585052    0.563378    0.566762   
       64              0.702768           0.717181    0.679852    0.701330   
4      4               0.128295           0.140088    0.121995    0.135465   
       8               0.201413           0.183462    0.191205    0.178176   
       16              0.410834           0.417641    0.387653    0.399347   
       32              0.587756           0.574388    0.569461    0.557336   
       64              0.697144           0.645337    0.673576    0.631637   

              VPCs (Farkas)  Root Cuts (None)  Root Cuts (New)  \
degree terms                                                     
1      4           0.146750          0.375367         0.375368   
       8           0.172525          0.375367         0.375368   
       16          0.371150          0.375367         0.419870   
       32          0.520394          0.375367         0.583886   
       64          0.625882          0.350652         0.678752   
2      4           0.132386          0.393164         0.393166   
       8           0.154641          0.393164         0.393169   
       16          0.321545          0.393164         0.440117   
       32          0.145424          0.377018         0.569217   
       64          0.248074          0.377061         0.683401   
4      4           0.107172          0.379682         0.379682   
       8           0.125617          0.379682         0.379692   
       16          0.260801          0.379682         0.425656   
       32          0.327513          0.368366         0.572924   
       64          0.324421          0.347143         0.675350   

              Root Cuts (Old)  Root Cuts (Farkas)  instance  problem number  
degree terms                                                                 
1      4             0.375368            0.375641         1              20  
       8             0.375368            0.375641         1              20  
       16            0.418322            0.401795         1              20  
       32            0.600594            0.549012         1              20  
       64            0.701570            0.630080         1              15  
2      4             0.393165            0.392988         1              20  
       8             0.393169            0.392987         1              20  
       16            0.430298            0.400688         1              20  
       32            0.579137            0.398811         1              11  
       64            0.701854            0.416500         1               5  
4      4             0.379683            0.379669         1              30  
       8             0.379683            0.379669         1              30  
       16            0.430429            0.395827         1              30  
       32            0.563483            0.466667         1              24  
       64            0.634034            0.494900         1              16

In [27]:
# now break it down by type of perturbation
bound_df.groupby(["degree", "terms", "perturbation"]).agg(aggregations)

Disjunction (New)  Disjunction (Old)  VPCs (New)  \
degree terms perturbation                                                     
1      4     matrix                 0.142168           0.150770    0.142168   
             rhs                    0.153537           0.162528    0.153537   
       8     matrix                 0.185304           0.181475    0.184879   
             rhs                    0.203018           0.198952    0.201168   
       16    matrix                 0.419783           0.406027    0.400019   
             rhs                    0.405154           0.431074    0.390385   
       32    matrix                 0.584615           0.581523    0.575790   
             rhs                    0.602998           0.621555    0.587463   
       64    matrix                 0.700192           0.714200    0.669112   
             rhs                    0.714568           0.721066    0.686717   
2      4     matrix                 0.134363           0.145849    0.134363   
             rhs                    0.130701           0.150435    0.130701   
       8     matrix                 0.182964           0.180907    0.182562   
             rhs                    0.195544           0.187415    0.191902   
       16    matrix                 0.444094           0.401658    0.411937   
             rhs                    0.407285           0.408866    0.387856   
       32    matrix                 0.592707           0.594560    0.579099   
             rhs                    0.419190           0.489968    0.406164   
       64    matrix                 0.682254           0.713373    0.660322   
             rhs                    0.784825           0.732415    0.757973   
4      4     matrix                 0.139393           0.151215    0.137208   
             objective              0.124318           0.135267    0.123691   
             rhs                    0.121174           0.133781    0.105085   
       8     matrix                 0.208014           0.183451    0.202791   
             objective              0.179792           0.169721    0.177967   
             rhs                    0.216435           0.197213    0.192858   
       16    matrix                 0.450489           0.474095    0.420520   
             objective              0.406395           0.392411    0.384628   
             rhs                    0.375617           0.386419    0.357811   
       32    matrix                 0.600723           0.582691    0.573395   
             objective              0.567896           0.572783    0.550843   
             rhs                    0.603565           0.565994    0.595247   
       64    matrix                 0.706210           0.615028    0.673700   
             objective              0.702128           0.707687    0.681692   
             rhs                    0.667382           0.560470    0.654393   

                           VPCs (Old)  VPCs (Farkas)  Root Cuts (None)  \
degree terms perturbation                                                
1      4     matrix          0.150770       0.143268          0.357151   
             rhs             0.162528       0.150232          0.393583   
       8     matrix          0.180492       0.172660          0.357151   
             rhs             0.198529       0.172390          0.393583   
       16    matrix          0.383112       0.357702          0.357151   
             rhs             0.409077       0.384599          0.393583   
       32    matrix          0.577796       0.520822          0.357151   
             rhs             0.612720       0.519965          0.393583   
       64    matrix          0.698008       0.585405          0.344333   
             rhs             0.702986       0.686597          0.360131   
2      4     matrix          0.145849       0.130199          0.378275   
             rhs             0.150435       0.134573          0.408053   
       8     matrix          0.180362       0.151138          0.378

## Make time table

In [28]:
def optimality_gap(df, generator):
    return abs(df[f"primalBound {generator}"] - df[f"dualBound {generator}"]) / \
        (df[f"primalBound {generator}"])

In [30]:
# additional filtering for dataframe on run time
fields = ["terminationTime New", "terminationTime Old",
          "terminationTime Farkas", "terminationTime None",
          "vpcGenerationTime New", "vpcGenerationTime Old",
          "vpcGenerationTime Farkas"]
mask = mask & (optimality_gap(df, "New") < .001)
time_df = df.loc[mask, group_cols + id_cols + fields]

# define aggregating operations
aggregations = {f: gmean for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

# get gap closed by degree and term
time_df.groupby(["degree", "terms"]).agg(aggregations)

terminationTime New  \
degree terms                                    
1      4                             0.082149   
       8                             0.088896   
       16                            0.118614   
       32                            0.142742   
       64                            0.236724   
2      4                             0.081829   
       8                             0.090376   
       16                            0.109113   
       32                            0.152466   
       64                            0.245091   
4      4                             0.077295   
       8                             0.085147   
       16                            0.107091   
       32                            0.143661   
       64                            0.228488   

              terminationTime Old  terminationTime Farkas  \
degree terms                                                            
1      4                             0.075293                0.074579   
       8                             0.082664                0.075433   
       16                            0.090929                0.081727   
       32                            0.099873                0.084515   
       64                            0.145734                0.105850   
2      4                             0.074570                0.074114   
       8                             0.083643                0.077157   
       16                            0.090833                0.081053   
       32                            0.106473                0.086641   
       64                            0.153824                0.101180   
4      4                             0.070546                0.069129   
       8                             0.076562                0.072576   
       16                            0.082199                0.074600   
       32                            0.101781                0.080990   
       64                            0.131550                0.094016   

              terminationTime None  vpcGenerationTime New  \
degree terms                                                            
1      4                  0.072781                           0.007855   
       8                  0.073828                           0.014594   
       16                 0.073419                           0.031845   
       32                 0.072333                           0.060969   
       64                 0.080187                           0.135473   
2      4                  0.072896                           0.007992   
       8                  0.072472                           0.015194   
       16                 0.072974                           0.030087   
       32                 0.076568                           0.063180   
       64                 0.084765                           0.141347   
4      4                  0.068587                           0.008114   
       8                  0.068313                           0.014615   
       16                 0.068519                           0.031693   
       32                 0.074481                           0.062364   
       64                 0.075102                           0.134850   

              vpcGenerationTime Old  vpcGenerationTime Farkas  \
degree terms                                                                
1      4                               0.003499                  0.000955   
       8                               0.005905                  0.001737   
       16                              0.011508                  0.003682   
       32                              0.024939                  0.008320   
       64                              0.052731                  0.017102   
2      4                               0.003496                  0.000953   
       8                               0.005927                  0.001714   
       16                              0.011257    

# Make node table

In [31]:
# additional filtering for dataframe on nodes processed
fields = ["nodes New", "nodes Old", "nodes Farkas",
          "nodes None"]
node_df = df.loc[mask, group_cols + id_cols + fields]

# define aggregating operations
aggregations = {f: gmean for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

# get gap closed by degree and term
node_df.groupby(["degree", "terms"]).agg(aggregations)

nodes New  nodes Old  nodes Farkas  \
degree terms                                                               
1      4                 123.943943             127.547147    126.784684   
       8                 120.894207             128.003228    126.533493   
       16                142.357123             139.870809    131.859392   
       32                131.999898             129.274357    138.860756   
       64                210.442692             224.446113    210.802292   
2      4                 101.316528             100.035059    104.184061   
       8                 102.845944             110.419438    104.892571   
       16                103.248648             109.520236    111.334298   
       32                152.399810             137.872512    141.574821   
       64                188.089675             192.589719    144.362707   
4      4                  98.318233              95.654156    105.645518   
       8                 102.205233              99.923366    103.413042   
       16                108.027910             104.414188    100.016728   
       32                121.547950             126.924927    117.847462   
       64                188.418781             166.935290    157.179545   

              nodes None  instance  problem number  
degree terms                                        
1      4      125.868236         1              20  
       8      125.868236         1              20  
       16     125.868236         1              20  
       32     125.868236         1              20  
       64     160.981302         1              15  
2      4      100.094918         1              20  
       8      100.094918         1              20  
       16     100.094918         1              20  
       32     127.852047         1              11  
       64     141.919384         1               5  
4      4      105.918433         1              30  
       8      105.918433         1              30  
       16     105.918433         1              30  
       32     110.455700         1              24  
       64     137.678718         1              16

# Make iteration table

In [32]:
# additional filtering for dataframe on nodes processed
fields = ["iterations New", "iterations Old",
          "iterations Farkas", "iterations None"]
node_df = df.loc[mask, group_cols + id_cols + fields]

# define aggregating operations
aggregations = {f: gmean for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

# get gap closed by degree and term
node_df.groupby(["degree", "terms"]).agg(aggregations)

iterations New  iterations Old  \
degree terms                                                           
1      4                     1798.188090                 1814.059314   
       8                     1783.224731                 1850.475198   
       16                    1979.231271                 1978.413887   
       32                    2021.916829                 1996.589398   
       64                    2633.624864                 2694.937924   
2      4                     1662.525326                 1655.002251   
       8                     1689.313196                 1742.663946   
       16                    1787.585002                 1815.426753   
       32                    2130.767721                 2114.773958   
       64                    2402.793442                 2530.688121   
4      4                     1614.753744                 1610.867764   
       8                     1642.588197                 1632.075113   
       16                    1742.200952                 1722.992129   
       32                    1835.878601                 1898.956858   
       64                    2404.923798                 2249.397305   

              iterations Farkas  iterations None  instance  problem number  
degree terms                                                                
1      4            1781.239575      1780.572140         1              20  
       8            1814.508533      1780.572140         1              20  
       16           1893.659317      1780.572140         1              20  
       32           2092.171422      1780.572140         1              20  
       64           2611.891972      1998.332159         1              15  
2      4            1678.598486      1645.028391         1              20  
       8            1688.397645      1645.028391         1              20  
       16           1786.328898      1645.028391         1              20  
       32           1944.333956      1842.540009         1              11  
       64           1979.392333      1903.189518         1               5  
4      4            1651.110426      1625.089754         1              30  
       8            1633.815546      1625.089754         1              30  
       16           1648.201424      1625.089754         1              30  
       32           1817.146783      1632.957580         1              24  
       64           2078.586444      1817.052427         1              16